In [1]:
import pandas as pd 
import numpy as np 

In [2]:

def read_csv_to_dataframe(file_path):
    try:
        # Read the CSV file into a Pandas DataFrame
        dataframe = pd.read_csv(file_path)
        return dataframe
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [3]:
patients_df = read_csv_to_dataframe("data/patients.csv")
prescriptions_df = read_csv_to_dataframe("data/prescriptions.csv")
inputevents_df = read_csv_to_dataframe("data/inputevents.csv")
procedureevents_df = read_csv_to_dataframe("data/procedureevents.csv")
d_icd_diagnoses_df = read_csv_to_dataframe("data/d_icd_diagnoses.csv")
triage_df = read_csv_to_dataframe("data/triage.csv")
vitalsign_df = read_csv_to_dataframe("data/vitalsign.csv")

/var/folders/fg/2gvx4j5s2msf761rcjqbxnvw0000gn/T/ipykernel_30063/723778750.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(file_path)


In [4]:
prescriptions_df.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route'],
      dtype='object')

In [5]:
inputevents_df

KeyboardInterrupt: 

In [ ]:
d_icd_diagnoses_df

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [6]:
data_pt = pd.merge(patients_df, triage_df, on='subject_id', how='inner')

In [ ]:
data_pt

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,97.8,87.0,14.0,97.0,71.0,43.0,7,2.0,Hypotension
1,10000032,F,52,2180,2014 - 2016,2180-09-09,33258284,98.4,70.0,16.0,97.0,106.0,63.0,0,3.0,"Abd pain, Abdominal distention"
2,10000032,F,52,2180,2014 - 2016,2180-09-09,35968195,99.4,105.0,18.0,96.0,106.0,57.0,10,3.0,"n/v/d, Abd pain"
3,10000032,F,52,2180,2014 - 2016,2180-09-09,38112554,98.9,88.0,18.0,97.0,116.0,88.0,10,3.0,Abdominal distention
4,10000032,F,52,2180,2014 - 2016,2180-09-09,39399961,98.7,77.0,16.0,98.0,96.0,50.0,13,2.0,"Abdominal distention, Abd pain, LETHAGIC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425082,19999784,M,57,2119,2017 - 2019,NaN,37972930,98.0,91.0,16.0,99.0,148.0,90.0,5,2.0,Abnormal MRI
425083,19999828,F,46,2147,2017 - 2019,NaN,30712109,98.1,83.0,18.0,100.0,107.0,75.0,8,2.0,"Abd pain, Wound eval"
425084,19999828,F,46,2147,2017 - 2019,NaN,32917002,96.6,112.0,18.0,100.0,110.0,82.0,4,2.0,"Abd pain, Wound eval"
425085,19999914,F,49,2158,2017 - 2019,NaN,32002659,99.5,81.0,10.0,100.0,93.0,55.0,0,2.0,"Altered mental status, Substance use"


In [8]:
data_pv = pd.merge(patients_df, vitalsign_df, on='subject_id', how='inner')

In [ ]:
data_pv

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:36:00,NaN,83.0,24.0,97.0,90.0,51.0,NaN,0
1,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:43:00,NaN,85.0,22.0,98.0,76.0,39.0,NaN,0
2,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:45:00,NaN,84.0,22.0,97.0,75.0,39.0,NaN,0
3,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 17:56:00,NaN,84.0,20.0,99.0,86.0,51.0,NaN,NaN
4,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564605,19999828,F,46,2147,2017 - 2019,NaN,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0,NaN,NaN
1564606,19999914,F,49,2158,2017 - 2019,NaN,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0,NaN,0
1564607,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 19:40:00,NaN,112.0,18.0,NaN,118.0,83.0,NaN,NaN
1564608,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 20:11:00,NaN,111.0,18.0,NaN,123.0,82.0,NaN,unable


In [9]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import pairwise_distances_argmin_min

In [ ]:
data_pv

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:36:00,NaN,83.0,24.0,97.0,90.0,51.0,NaN,0
1,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:43:00,NaN,85.0,22.0,98.0,76.0,39.0,NaN,0
2,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 16:45:00,NaN,84.0,22.0,97.0,75.0,39.0,NaN,0
3,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 17:56:00,NaN,84.0,20.0,99.0,86.0,51.0,NaN,NaN
4,10000032,F,52,2180,2014 - 2016,2180-09-09,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564605,19999828,F,46,2147,2017 - 2019,NaN,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0,NaN,NaN
1564606,19999914,F,49,2158,2017 - 2019,NaN,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0,NaN,0
1564607,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 19:40:00,NaN,112.0,18.0,NaN,118.0,83.0,NaN,NaN
1564608,19999987,F,57,2145,2011 - 2013,NaN,34731548,2145-11-02 20:11:00,NaN,111.0,18.0,NaN,123.0,82.0,NaN,unable


In [10]:
data_pi = pd.merge(data_pv, inputevents_df, on='subject_id', how='inner')

: 

: 